In [1]:
import cv2
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
import xml.etree.ElementTree as ET
import os
import torch


In [2]:
os.chdir(r"C:\Users\Lucas\Desktop\vacantion classes\Yolo project\classes")
from CNN import CNN_model #import my class CNN

In [3]:
os.chdir(r"C:\Users\Lucas\Desktop\vacantion classes\Yolo project\data\Pascal_VOC\train")

In [4]:
path = r"C:\Users\Lucas\Desktop\vacantion classes\Yolo project\data\Pascal_VOC\train"

xml_list = []
for filename in os.listdir(path):
    if not filename.endswith('.xml'): continue
    fullname = os.path.join(path, filename)
    xml_list.append(fullname)

In [5]:
my_CNN = CNN_model()
my_CNN.get_annotation(xml_list)
my_CNN.encode_pictures()

In [6]:
os.chdir(r"C:\Users\Lucas\Desktop\vacantion classes\Yolo project")
my_CNN.df.to_json(r"data\annot_df.json",orient="records")
torch.save(my_CNN.encoded_picture_annot, r"data\encoded_picture.pt")

In [7]:
df = pd.read_json(r"data\annot_df.json")
df

,file_name,ID_class,x_center,y_center,width,height
0,C:\Users\Lucas\Desktop\vacantion classes\Yolo ...,0,0.479000,0.464413,0.542000,0.373665
1,C:\Users\Lucas\Desktop\vacantion classes\Yolo ...,0,0.330000,0.375445,0.128000,0.124555
2,C:\Users\Lucas\Desktop\vacantion classes\Yolo ...,1,0.408000,0.727758,0.036000,0.174377
3,C:\Users\Lucas\Desktop\vacantion classes\Yolo ...,1,0.070000,0.759786,0.036000,0.174377
4,C:\Users\Lucas\Desktop\vacantion classes\Yolo ...,0,0.508000,0.505464,0.980000,0.426230
...,...,...,...,...,...,...
31351,C:\Users\Lucas\Desktop\vacantion classes\Yolo ...,1,0.740000,0.541333,0.500000,0.917333
31352,C:\Users\Lucas\Desktop\vacantion classes\Yolo ...,1,0.342988,0.635000,0.326220,0.390000
31353,C:\Users\Lucas\Desktop\vacantion classes\Yolo ...,1,0.742378,0.558000,0.149390,0.212000
31354,C:\Users\Lucas\Desktop\vacantion classes\Yolo ...,1,0.512012,0.485000,0.681682,0.618000


In [8]:
df_encoded = torch.load(r"data\encoded_picture.pt")
df_encoded

,file_name,encoded_grid
1,C:\Users\Lucas\Desktop\vacantion classes\Yolo ...,"[[[tensor(0.), tensor(0.), tensor(0.), tensor(..."
2,C:\Users\Lucas\Desktop\vacantion classes\Yolo ...,"[[[tensor(0.), tensor(0.), tensor(0.), tensor(..."
3,C:\Users\Lucas\Desktop\vacantion classes\Yolo ...,"[[[tensor(0.), tensor(0.), tensor(0.), tensor(..."
4,C:\Users\Lucas\Desktop\vacantion classes\Yolo ...,"[[[tensor(0.), tensor(0.), tensor(0.), tensor(..."
5,C:\Users\Lucas\Desktop\vacantion classes\Yolo ...,"[[[tensor(0.), tensor(0.), tensor(0.), tensor(..."
...,...,...
13295,C:\Users\Lucas\Desktop\vacantion classes\Yolo ...,"[[[tensor(0.), tensor(0.), tensor(0.), tensor(..."
13296,C:\Users\Lucas\Desktop\vacantion classes\Yolo ...,"[[[tensor(0.), tensor(0.), tensor(0.), tensor(..."
13297,C:\Users\Lucas\Desktop\vacantion classes\Yolo ...,"[[[tensor(0.), tensor(0.), tensor(0.), tensor(..."
13298,C:\Users\Lucas\Desktop\vacantion classes\Yolo ...,"[[[tensor(0.), tensor(0.), tensor(0.), tensor(..."


In [9]:
def yolo_loss(y_true,y_pred):
    return

In [10]:
import cv2
import numpy as np
import torch

S = 7
B = 2
C = len(my_CNN.classes)  # ton nombre de classes

X_train = []
Y_train = []

for idx, row in my_CNN.encoded_picture_annot.iterrows():
    if idx > 1000:
        break
    # Charger l'image
    img_path = row['file_name']
    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, (448,448))  # taille YOLO standard
    img = img / 255.0  # normalisation
    X_train.append(img)

    # Convertir le y_true de tensor à numpy
    y_tensor = row['encoded_grid']  # c'est un tensor ou liste de tensors
    if isinstance(y_tensor, torch.Tensor):
        y_numpy = y_tensor.numpy()
    else:
        # Si c'est une liste de listes de tensors
        y_numpy = np.array([[t.item() for t in line] for line in y_tensor])
    Y_train.append(y_numpy)

X_train = np.array(X_train, dtype=np.float32)
Y_train = np.array(Y_train, dtype=np.float32)

print(X_train.shape, Y_train.shape)

(1000, 448, 448, 3) (1000, 7, 7, 30)


## Model

In [11]:
model = my_CNN.Yolo1(C=20)  # your model
model.compile(optimizer='adam', 
              loss=lambda y_true, y_pred: my_CNN.yolo_loss_tf(y_true, y_pred, S=7, B=2, C=len(my_CNN.classes)),
              metrics=[])

In [12]:
tf.debugging.set_log_device_placement(True)

In [13]:
from tensorflow.python.client import device_lib 
print(device_lib.list_local_devices())
tf.debugging.set_log_device_placement(True)

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 16534968195606399476
]


In [14]:
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    for gpu in gpus:
        tf.config.experimental.set_memory_growth(gpu, True)

In [15]:
import tensorflow as tf
print(tf.config.list_physical_devices('GPU'))

[]


In [16]:
history = model.fit(X_train, Y_train, epochs=10, batch_size=8, validation_split=0.1)

Epoch 1/10
113/113 [==============================] - 341s 3s/step - loss: nan - val_loss: nan
Epoch 2/10
113/113 [==============================] - 301s 3s/step - loss: nan - val_loss: nan
Epoch 3/10
113/113 [==============================] - 330s 3s/step - loss: nan - val_loss: nan
Epoch 4/10
113/113 [==============================] - 329s 3s/step - loss: nan - val_loss: nan
Epoch 5/10
113/113 [==============================] - 298s 3s/step - loss: nan - val_loss: nan
Epoch 6/10
113/113 [==============================] - 302s 3s/step - loss: nan - val_loss: nan
Epoch 7/10
113/113 [==============================] - 310s 3s/step - loss: nan - val_loss: nan
Epoch 8/10
113/113 [==============================] - 300s 3s/step - loss: nan - val_loss: nan
Epoch 9/10
113/113 [==============================] - 297s 3s/step - loss: nan - val_loss: nan
Epoch 10/10
113/113 [==============================] - 296s 3s/step - loss: nan - val_loss: nan
